In [ ]:
import sys, os, glob, yaml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
import seaborn as sns
import trackml.dataset

In [ ]:
import torch
from torch_geometric.data import Data
import itertools

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
sys.path.append('..')

In [ ]:
from src.drawing import detector_layout, draw_proc_event
from src.utils_math import polar_to_cartesian

## _(A.) Visualize Test Data_

Inspect easy to difficult to reconstruct tracks. Choose 10 such events and use `score_cut=0.25`

In [ ]:
inputdir="../run_all/fwp_gnn_processed/pred"
proc_files = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
feature_data = torch.load(proc_files[0], map_location=device)

In [ ]:
feature_data.event_file[-4:]

In [ ]:
event_id = int(os.path.basename(feature_data.event_file)[-4:])

In [ ]:
event_id

## _(B.) Inspection on Track Duplication_

- lets plot two reconstructed events, one is `clean` and the other a little `complex`
- see an event with duplicated tracks, find out why they are duplicated.

In [ ]:
# get list of true tracks (test dataset)
inputdir="../run_all/fwp_gnn_processed/pred"
true_trkx = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
# get list of reco tracks (test dataset)
inputdir="../run_all/fwp_gnn_segmenting/seg"
reco_trkx = sorted(glob.glob(os.path.join(inputdir, "*")))

### _(1.) - Plot True Event (Clean)_

In [ ]:
# The Good Event
event_idx = 15

In [ ]:
# true events
true_event = torch.load(true_trkx[event_idx], map_location=device)

In [ ]:
true_event.event_file[-4:]

In [ ]:
np.unique(true_event.pid)

In [ ]:
# hit features
r, phi, ir = true_event.x.T

In [ ]:
# spatial coordinate transform
x, y = polar_to_cartesian(r.detach().numpy(), phi.detach().numpy())

In [ ]:
# compensate scaling
ir = ir.detach().numpy()*100

In [ ]:
# plot true event
fig, ax = detector_layout(figsize=(10,10))
e_id = int(true_event.event_file[-4:])
p_ids = np.unique(true_event.pid)

for pid in p_ids:
    idx = true_event.pid == pid
    ax.plot(x[idx], y[idx], '-', linewidth=1.5)
    ax.scatter(x[idx], y[idx], s=(ir[idx]*50), label='particle_id: {}'.format(int(pid)))

ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
fig.savefig("clean_true.png")

### _(2.) - Plot Reco Event (Clean)_

In [ ]:
# lets plot reconstructed true event, i.e. event = 96
reco_event = torch.load(reco_trkx[event_idx], map_location=device)

In [ ]:
reco_event.head()

Now, we've only the `hit_id` and `track_id`, what are corresponding spatial coordinates (x,y coordinates)? How about using `hit_id` to filter features from the true event?

Note, the `hit_id`, `track_id` in reco data are equiv. to `hid`, `pid` in true data.

In [ ]:
# plot reco event, here, (x,y,ir) comes from ture event. but idx is from reco event
fig, ax = detector_layout(figsize=(10,10))
e_id = e_id
t_ids = np.unique(reco_event.track_id)
for tid in t_ids:
    idx = reco_event.track_id == tid
    ax.plot(x[idx], y[idx], '-', linewidth=1.5)
    ax.scatter(x[idx], y[idx], s=(ir[idx]*50), label='particle_id: {}'.format(tid))
    
ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
fig.savefig("clean_reco.png")

In [ ]:
inputdir="../run_all/fwp_gnn_segmenting/eval"

In [ ]:
evtid = int(os.path.split(reco_trkx[event_idx])[1])

In [ ]:
print("evtid: {}".format(evtid))

In [ ]:
!./eval_reco_evt.sh $e_id

In [ ]:
# path to HDF5 Store
hdf5_path = inputdir+"/{}_particles.h5".format(e_id)
print(hdf5_path)

In [ ]:
df = pd.read_hdf(hdf5_path)
print("Length of Data: {}".format(len(df)))

In [ ]:
df.head()

In [ ]:
# Read the Summary of Evaluation
sum_path = inputdir+"/{}_summary.txt".format(e_id)
print(sum_path)

In [ ]:
with open(sum_path) as f:
    contents = f.read()
    print(contents)

### _(3.) - Plot True Event (Complex)_

In [ ]:
# The Good Event
event_idx = 51

In [ ]:
# true events
true_event = torch.load(true_trkx[event_idx], map_location=device)

In [ ]:
true_event.event_file[-4:]

In [ ]:
np.unique(true_event.pid)

In [ ]:
# hit features
r, phi, ir = true_event.x.T

In [ ]:
# spatial coordinate transform
x, y = polar_to_cartesian(r.detach().numpy(), phi.detach().numpy())

In [ ]:
# compensate scaling
ir = ir.detach().numpy()*100

In [ ]:
# plot true event
fig, ax = detector_layout(figsize=(10,10))
e_id = int(true_event.event_file[-4:])
p_ids = np.unique(true_event.pid)

for pid in p_ids:
    idx = true_event.pid == pid
    ax.plot(x[idx], y[idx], '-', linewidth=1.5)
    ax.scatter(x[idx], y[idx], s=(ir[idx]*50), label='particle_id: {}'.format(int(pid)))

ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
# fig.savefig("{}_complex_true.png".format(e_id))

### _(4.) - Plot Reco Event (Complex)_

In [ ]:
# lets plot reconstructed true event, i.e. event = 96
reco_event = torch.load(reco_trkx[event_idx], map_location=device)

In [ ]:
reco_event.head()

Now, we've only the `hit_id` and `track_id`, what are corresponding spatial coordinates (x,y coordinates)? How about using `hit_id` to filter features from the true event?

Note, the `hit_id`, `track_id` in reco data are equiv. to `hid`, `pid` in true data.

In [ ]:
# plot reco event, here, (x,y,ir) comes from ture event. but idx is from reco event
fig, ax = detector_layout(figsize=(10,10))
e_id = e_id
t_ids = np.unique(reco_event.track_id)
for tid in t_ids:
    idx = reco_event.track_id == tid
    ax.plot(x[idx], y[idx], '-', linewidth=1.5)
    ax.scatter(x[idx], y[idx], s=(ir[idx]*50), label='particle_id: {}'.format(tid))
    
ax.set_title('Azimuthal View of STT, EventID # {}'.format(e_id))
ax.legend(fontsize=10, loc='best')
fig.tight_layout()
# fig.savefig("{}_complex_reco.png".format(e_id))

In [ ]:
!./eval_reco_evt.sh $e_id

In [ ]:
# Read the Summary of Evaluation
sum_path = inputdir+"/{}_summary.txt".format(e_id)
print(sum_path)

In [ ]:
with open(sum_path) as f:
    contents = f.read()
    print(contents)

## _Unassigned Hits_

What is _`track_id=-1`_ contains?

The _hits_ not assigned to any track are collected under `track_id=-1`.

In [ ]:
dup_hids = reco_event.query("track_id==-1")["hit_id"]

In [ ]:
dup_hids.values